In [ ]:
from heapq import merge as merge_sorted_lists
from multiprocessing import Manager, Pool
from modules.sorting import merge_sort, merge_sort_v2, merge_sort_v3, append_sorted_list
from modules.helpers import generate_random_int_list, generate_random_float_list
from modules.timing import function_timing
import matplotlib.pyplot as plt
import time
import timeit

In [ ]:
def threaded_merge_sort(array, threads):
    
    # start time
    start = time.time()
    
    # Create multiprocessing objects
    thread_manager = Manager()
    result_manager = thread_manager.list()
    pool = Pool(threads)
    
    # Add asynchronous processes to pool
    for i in range(threads):
        pool.apply_async(append_sorted_list, (result_manager, array[i::threads]))
    
    # Close pool, start, and wait till all processes are finished
    pool.close()
    pool.join()
    
    # Merge sorted lists back into one sorted list
    result = list(merge_sorted_lists(*result_manager))
    
    # Save run time
    timer = time.time() - start
    
    # Merge sort higher dimensional list with module merge from heapq
    return result, timer

In [ ]:
if __name__ == '__main__':
    
    # Random list with floats or ints
    array = generate_random_float_list(100000)
    
    # Timings
    results = []
    print("Python Sorted time: " + str(timeit.timeit(lambda: sorted(array), number=1)))
    for i in range(1, 9):
        sorted_array, timing = threaded_merge_sort(array, i)
        results.append(timing)
        print(f"{i} thread(s): {str(timing)} | Equal to sorted(array): {sorted_array == sorted(array)}")
        
    # Plot timings
    plt.figure(figsize=(12,6))
    plt.plot(list(range(1, len(results)+1)), results)
    plt.xlabel("number of threads")
    plt.ylabel("runtime in seconds")
    plt.xticks(list(range(1,len(results)+1)))